## Overview

This script was created to perform the ETL process for building a database out of D&D and folklore creatures. The ETL process was executed with the use of Python libraries and PostgreSQL. The details of each step are explained throughout this file.

In [ ]:
# import dependencies
import pandas as pd
import unicodedata
import re
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import Session
from sqlalchemy.dialects import postgresql
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# import the PostgreSQL confidential values
from config import postgresql_key, postgresql_port, postgresql_host, postgresql_db, postgresql_user

## Extract

#### Web Scraping
The first data source was scraped from a [Wikipedia multi-page table](https://en.wikipedia.org/wiki/List_of_legendary_creatures_(A)) that alphabetically lists folklore/mythological creatures along with their origin and a short description. Due to the nature of this algorithm, we decided it was most efficient to format the data as it was scraped. We encountered issues where the list items had inconsistent formatting with their anchor elements so we had to resort to pulling the displayed text then delimiting it by parentheses in order to still get the desired information. After storing this data into string variables, we had to convert any special and accented characters to standard letters. We used the ``strip_accents`` function to this effect. After the data was properly formatted it was stored in a DataFrame for later use.

#### CSV
The second data source was a CSV downloaded from [Kaggle](https://www.kaggle.com/datasets/mrpantherson/dnd-5e-monsters). We imported the CSV using Pandas native ``read_csv`` method.

In [ ]:
# removes accented characters from a string
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

# set up the splinter service
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = True)

# specify the initial web browser URL
web_url = "https://en.wikipedia.org/wiki/List_of_legendary_creatures_(A)"

# send the browser instance to the provided URL
browser.visit(web_url)

# initialize the destination lists
names = []
origins = []
descriptions = []

# define the alphabetical list for browser navigation
alphabet = ["B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "A"]

# iterate through the web pages and scrape the content into the destination lists
for letter in alphabet:
    
    # define the BeautifulSoup instance
    soup = bs(browser.html, "html.parser")
    
    # retrieve the creature list
    creatures = soup.body.find("div", class_ = "mw-body-content mw-content-ltr").find_all("ul")[1].find_all("li")
    
    # split the list items into name and origin then store into lists
    for creature in creatures:
        
        # store the whole list item text
        myStr = creature.text
        
        # make sure there are parentheses to delimit with
        if "(" in myStr:
            
            # split the text by parentheses
            split0 = myStr.split("(")
            split1 = split0[1].split(")")
            
            # extract the relevant information
            name = strip_accents(split0[0].strip()).lower()
            origin = strip_accents(split1[0].strip()).lower()
            description = strip_accents(split1[len(split1) - 1].strip()[1:].strip()).lower()
            
            # store the information into lists
            names.append(name)
            origins.append(origin)
            descriptions.append(description)
    
    # advance to the next page
    browser.links.find_by_href(f"/wiki/List_of_legendary_creatures_({letter})").click()

browser.quit()

# store the lists in a dataframe
folklore_creatures_df = pd.DataFrame({
    "names": names,
    "origins": origins,
    "descriptions": descriptions})

# import dnd monster csv into a Pandas dataframe
csv_file = "Resources/dnd_monsters.csv"
dndmonster_df= pd.read_csv(csv_file)

## Transform

#### Web Scraping
This transformation process was completed within the web scraping algorithm for better efficiency.

#### CSV
This dataset contained more data than we needed so we decided to remove some columns after importing the CSV into a DataFrame. We also encountered some invalid data so some rows had to be dropped. The "cr" column contained data in the incorrect format so we had to perform some extra steps to replace ``string`` fractions into ``string`` decimals then convert the whole column into the ``float`` type. After we had all the data we wanted into the correct format, we renamed the columns to better represent the data they contain. We then stored this transformed data into a new DataFrame for later use.

In [ ]:
# trim out unnecessary columns
clean_monster_df = dndmonster_df[["name","cr","type","size","ac","hp","align"]]

# remove invalid data
clean_monster_df.dropna()

# convert text representations of fractions into decimals
clean_monster_df["cr"] = clean_monster_df["cr"].apply(lambda s: re.sub(r"1/4","0.25", str(s)))
clean_monster_df["cr"] = clean_monster_df["cr"].apply(lambda s: re.sub(r"1/2","0.50", str(s)))
clean_monster_df["cr"] = clean_monster_df["cr"].apply(lambda s: re.sub(r"1/8","0.125", str(s)))

# convert 'challenge rating' column into float from string
clean_monster_df["cr"] = clean_monster_df["cr"].astype(float)

# rename columns
clean_monster_df = clean_monster_df.rename(columns = {
    "cr": "challenge_rating",
    "ac": "armor_class",
    "hp": "hit_points",
    "align": "alignment"})

# confirm data types
clean_monster_df.dtypes

## Load

Before we could load our data into a database, we had to create one and its constituent tables. To do this, we used [QuickDBD](https://app.quickdatabasediagrams.com/#/) to design the ERD then exported the schema as an SQL file (``schema.sql``). A graphical view of the ERD is available [here](erd.png). After creating the schema script we ran it within the pgAdmin4 Query Tool for the database we created.

Once the data was properly formatted and the database was created, we were able to load the data into the PostgreSQL database as defined above. We first reflected the tables then stored these tables into their own objects. In order to transfer data to the database tables we had to create a SQLAlchemy ``Session`` then use the ``.add()`` method within an ``iterrow()`` loop. This process had to be ran twice; one for each DataFrame/table. Once the data was queued in the session we simply used the ``.commit()`` method to send the data.

In [ ]:
# create the engine
engine = create_engine(f"postgresql+psycopg2://{postgresql_user}:{postgresql_key}@{postgresql_host}/{postgresql_db}")

# create the base reflector
Base = automap_base()
Base.prepare(engine, reflect = True)

# define the tables
dnd_monsters_tbl = Base.classes.dnd_monsters
folklore_creatures_tbl = Base.classes.folklore_creatures

# create the session
session = Session(engine)

# add data from the clean_monster_df to the current session
for index, row in clean_monster_df.iterrows():
    session.add(dnd_monsters_tbl(name = row["name"], challenge_rating = row["challenge_rating"], type = row["type"], size = row["size"], armor_class = row["armor_class"], hit_points = row["hit_points"], alignment = row["alignment"]))

# add data from the folklore_creatures_df to the current session
for index, row in folklore_creatures_df.iterrows():
    session.add(folklore_creatures_tbl(name = row["names"], origin = row["origins"], description = row["descriptions"]))

# send the new data to the database then flush the session
session.commit()

## Example

To prove our data could be combined into one table for future analysis, we attempted an ``inner join`` on the ``name`` categories. This showed our data can be used to link folklore and mythological creatures with D&D statistics. We finally closed our session with the ``.close()`` method.

In [ ]:
# query the database for an inner join
results = session.query(*[dnd_monsters_tbl.name, dnd_monsters_tbl.challenge_rating, dnd_monsters_tbl.armor_class, dnd_monsters_tbl.hit_points, folklore_creatures_tbl.description])\
                .join(folklore_creatures_tbl, dnd_monsters_tbl.name == folklore_creatures_tbl.name)\
                .order_by(dnd_monsters_tbl.name.asc()).all()

# display the results as a collection of tuples
for row in results:
    print(row)

# close and release session resource
session.close()